In [14]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import datetime as dt

In [45]:
dataset = pd.read_csv('D:\Data Science\Dataset\data.csv\data.csv', encoding= 'unicode_escape')
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [46]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [39]:
dataset.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
# looking for the distribution 
data_num = dataset.select_dtypes("number")

a = 1
for x in data_num.columns:
    print("{}. {}".format(a, x))
    fig, axes = plt.subplots(2, 1)
    sns.histplot(dataset[x], kde= True, ax = axes[0])
    sns.barplot(data = dataset, x= x, ax= axes[1])
    plt.show()

    q1 = np.percentile(dataset[x], 25)
    q3 = np.percentile(dataset[x],75)
    iqr = q3 - q1
    upper_fence = q3 + 1.5 * iqr
    lower_fence = q3 - 1.5 * iqr
    outlier = list(dataset[(dataset[x] < lower_fence) | (dataset[x] > upper_fence)][x])
    print("Total outlier = {}".format(len(outlier)))
    a+=1
    print("-"*100)

## Data Preprocessing

In [47]:
# Convert to datetime
dataset['InvoiceDate'] = pd.to_datetime(dataset['InvoiceDate'])

# Exclude negative values dari dataset, pada kolom quantity dan unitprice terdapat nilai negative
dataset = dataset[(dataset['Quantity'] > 0) & (dataset['UnitPrice'] > 0)]

# Barang yang dikembalikan (memiliki nilai C pada StockCode) di drop
dataset = dataset[dataset['StockCode'] != 'C']

# drop duplicate 
dataset.drop_duplicates()

# drop null value 
dataset.dropna(inplace = True)

# Total_Price = UnitPrice * Quantity
dataset['Total_Price'] = dataset['UnitPrice'] * dataset['Quantity']

# customer id as int 
dataset['CustomerID'] = dataset['CustomerID'].astype(int)

## RFM Analysis

In [48]:
# Menentukan tanggal untuk melakukan analisis (patokan Recency)
dataset['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [49]:
now = dt.datetime(2011, 12, 10)

In [55]:
# RFM dataframe. groupby by customer id 
rfm = dataset.groupby('CustomerID').agg({'InvoiceDate' : lambda x: (now - x.max()).days,
                                         'InvoiceNo' : 'count',
                                         'Total_Price' : 'sum'})

# rename column name 
rfm.rename(columns= {'InvoiceDate' : 'Recency',
                     'InvoiceNo' : 'Frequency',
                     'Total_Price' : 'MonetaryValue'}, inplace = True)

In [56]:
rfm.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346,325,1,77183.60
12347,2,182,4310.00
12348,75,31,1797.24
12349,18,73,1757.55
12350,310,17,334.40


In [57]:
# dikelompokan berdasarkan quartile, semakin besar label semakin baik
rfm['R'] = pd.qcut(rfm['Recency'],5, labels= [5, 4, 3 ,2, 1])
rfm['F'] = pd.qcut(rfm['Frequency'], 5, labels= [1, 2, 3, 4, 5])
rfm['M'] = pd.qcut(rfm['MonetaryValue'], 5, labels= [1, 2, 3, 4, 5])

In [58]:
# menghitung RFM score berdasarkan nilai pada R + F + M 
rfm['RFM_Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

In [59]:
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score
CustomerID,,,,,,,
12346,325,1,77183.60,1,1,5,115
12347,2,182,4310.00,5,5,5,555
12348,75,31,1797.24,2,3,4,234
12349,18,73,1757.55,4,4,4,444
12350,310,17,334.40,1,2,2,122


# Segmentation

In [60]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [61]:
rfm['Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score,Segment
CustomerID,,,,,,,,
12346,325,1,77183.60,1,1,5,115,Hibernating
12347,2,182,4310.00,5,5,5,555,Champions
12348,75,31,1797.24,2,3,4,234,At Risk
12349,18,73,1757.55,4,4,4,444,Loyal Customers
12350,310,17,334.40,1,2,2,122,Hibernating


In [63]:
rfm.groupby('Segment').mean().sort_values('MonetaryValue')

,Recency,Frequency,MonetaryValue
Segment,,,
Promising,22.437500,7.758929,429.433929
About to Sleep,52.192547,15.987578,450.997174
Hibernating,209.251397,13.585661,536.535672
Need Attention,51.985366,41.731707,856.195854
Potential Loyalists,15.668699,34.936992,915.450409
At Risk,165.435852,56.859402,996.916872
Can't Loose,143.226190,181.642857,2370.705012
Loyal Customers,33.206854,157.106487,2845.732852
New Customers,6.421053,7.578947,3618.697018
